In [1]:
import os
import pandas as pd
import math
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
%matplotlib inline
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import langid

### References (Code used from the following)

* https://www.kaggle.com/code/vatsalmavani/music-recommendation-system-using-spotify-dataset/notebook
* https://towardsdatascience.com/how-to-build-an-amazing-music-recommendation-system-4cce2719a572
* https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset/
* https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs/

In [2]:
# The priority is to have many songs inside the final dataframe as possible, 
# even if a song may have missing features
# df1 has the most songs as a base
# genre is an important feature to keep which is what df2 has a feature for all of its songs
# df3 has more songs that both df1 and df2 lack so we will add those as well
# df4 also has a genre column for its songs we will try to merge those to the final df
# Eventually all dataframes will be merged into one.

df1 = pd.read_csv('songs_a.csv', low_memory=False)
df2 = pd.read_csv('songs_b.csv', low_memory=False)

df3 = pd.read_csv('songs_c.csv', low_memory=False)
df4 = pd.read_csv('songs_d.csv', low_memory=False)  

# df2 = df2[['track_id', 'track_genre']]
df4 = df4[['id', 'genre']]

df4['genre'] = df4['genre'].apply(lambda x: x.lower())

In [3]:
df2.dropna(inplace=True)
df2.drop(["Unnamed: 0","popularity"], axis=1, inplace=True)
df2['artists'] = df2['artists'].apply(lambda x: str(x.replace(";", ", ").split(", ")))
df2.rename(columns={"track_id": "id", "track_name":"name", "album_name":"album"}, inplace=True)
df4.rename(columns={"genre": "track_genre"}, inplace=True)
# df1[df1['artists'].str.contains("Stevie Wonder")]

In [4]:
df1.columns, df2.columns

(Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
        'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
        'key', 'loudness', 'mode', 'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
        'time_signature', 'year', 'release_date'],
       dtype='object'),
 Index(['id', 'artists', 'album', 'name', 'duration_ms', 'explicit',
        'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'time_signature', 'track_genre'],
       dtype='object'))

In [5]:
df1.dropna(axis=0, inplace=True)
df3.drop("popularity", axis=1, inplace=True)

In [6]:
cols_to_fill = ['name', 'artists', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']

In [7]:
df_ = df1.merge(df2, how="outer", on="id", suffixes=[None, "_y"])

# print(t.columns)
for c in cols_to_fill:
    df_[c].fillna(df_[f"{c}_y"], inplace=True)

df_.drop(df_.filter(regex='_y$').columns, axis=1, inplace=True)

print(df_.shape, df_.columns)

(1158618, 25) Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date', 'track_genre'],
      dtype='object')


In [8]:
df_b = df3.merge(df4, how="left", on="id")
# print(df_b.shape, df_b.columns, df_b.columns.difference(df_.columns))

df_ = df_.merge(df_b, how="outer", on="id", suffixes=[None,"_y"])
# print(df_.shape, df_.columns)

In [9]:
df_.columns

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date', 'track_genre', 'valence_y',
       'year_y', 'acousticness_y', 'artists_y', 'danceability_y',
       'duration_ms_y', 'energy_y', 'explicit_y', 'instrumentalness_y',
       'key_y', 'liveness_y', 'loudness_y', 'mode_y', 'name_y',
       'release_date_y', 'speechiness_y', 'tempo_y', 'track_genre_y'],
      dtype='object')

In [10]:
cols_to_fill = ['name', 'artists', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'year', 'release_date']

cols_to_use = df_.columns[:25]

In [11]:
df_[cols_to_use].isnull().sum()

id                        0
name                 146075
album                255430
album_id             255430
artists              146075
artist_ids           255430
track_number         255430
disc_number          255430
explicit             146075
danceability         146075
energy               146075
key                  146075
loudness             146075
mode                 146075
speechiness          146075
acousticness         146075
instrumentalness     146075
liveness             146075
valence              146075
tempo                146075
duration_ms          146075
time_signature       255430
year                 255430
release_date         255430
track_genre         1190935
dtype: int64

In [12]:
for c in cols_to_fill:
    df_[c].fillna(df_[f"{c}_y"], inplace=True)

In [13]:
df_[cols_to_use].isnull().sum()

id                        0
name                      0
album                255430
album_id             255430
artists                   0
artist_ids           255430
track_number         255430
disc_number          255430
explicit                  0
danceability              0
energy                    0
key                       0
loudness                  0
mode                      0
speechiness               0
acousticness              0
instrumentalness          0
liveness                  0
valence                   0
tempo                     0
duration_ms               0
time_signature       255430
year                 101242
release_date         101242
track_genre         1190935
dtype: int64

In [14]:
df_.drop(df_.filter(regex='_y$').columns, axis=1, inplace=True)

In [15]:
df_.isnull().sum()

id                        0
name                      0
album                255430
album_id             255430
artists                   0
artist_ids           255430
track_number         255430
disc_number          255430
explicit                  0
danceability              0
energy                    0
key                       0
loudness                  0
mode                      0
speechiness               0
acousticness              0
instrumentalness          0
liveness                  0
valence                   0
tempo                     0
duration_ms               0
time_signature       255430
year                 101242
release_date         101242
track_genre         1190935
dtype: int64

In [18]:
df_.shape

(1305272, 25)

In [19]:
df_.drop(['track_number','disc_number'], axis=1, inplace=True)

In [20]:
df_.drop_duplicates(subset=['id'], inplace=True, keep='last')
df_.drop_duplicates(subset=['artist_ids','name'], inplace=True, keep='first')

In [21]:
df_['explicit'] = df_['explicit'].astype(bool)

In [22]:
df_.dtypes

id                   object
name                 object
album                object
album_id             object
artists              object
artist_ids           object
explicit               bool
danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms         float64
time_signature      float64
year                float64
release_date         object
track_genre          object
dtype: object

In [23]:
df_['track_genre'].value_counts()

track_genre
comedy       973
grindcore    950
sertanejo    932
malay        922
turkish      916
            ... 
latin        184
latino       164
reggae       157
alt-rock     139
edm          113
Name: count, Length: 113, dtype: int64

# Removing remixed songs (songs that are remixes / live performances)

In [24]:
df_ = df_[~df_['name'].str.contains("-|\(|\)")]

In [25]:
df_.name

0                              Testify
1                      Guerrilla Radio
2                     Calm Like a Bomb
3                            Mic Check
4                Sleep Now In the Fire
                      ...             
1305263         I Don't Think I'm Okay
1305264             Timeless Interlude
1305266                       Sunblind
1305268         NASTY GIRL / ON CAMERA
1305269    Halloweenie III: Seven Days
Name: name, Length: 941260, dtype: object

In [31]:
# Find the count of the first artist in a list
df_["artists"] = df_["artists"].apply(
    lambda x: x.replace("[", "")
    .replace("]", "")
    .replace("'", "")
    .replace(";", ", ")
    .split(", ")
)

In [32]:
df_['primary_artist'] = df_['artists'].apply(lambda x: x[0].replace('\"',''))
df_['secondary_artists'] = df_['artists'].apply(lambda x:", ".join(x[1:]))

In [33]:
df_["artist_ids"].fillna("[]",inplace=True)

In [34]:
df_["artist_ids"] = df_["artist_ids"].apply(
    lambda x: str(x).replace("[", "")
    .replace("]", "")
    .replace("'", "")
    .replace(";", ", ")
    .split(", ")
)

df_['primary_artist_id'] = df_['artist_ids'].apply(lambda x: x[0] if x[0] != '' else None)
# 175839 missing
df_['primary_artist_id'] = df_['primary_artist_id'].fillna(df_.groupby('primary_artist')['primary_artist_id'].transform("first"))
# 82235 missing

df_['primary_artist_id'].fillna("", inplace=True)

df_['secondary_artist_ids'] = df_['artist_ids'].apply(lambda x:", ".join(x[1:]))

### Filling other missing data using existing data

In [35]:
df_['temp'] = df_.apply(lambda x:f"{x['primary_artist_id']} {x['album']}", axis=1)
df_['album_id'].fillna(df_.groupby('temp')['album_id'].transform("first"), inplace=True)

In [36]:
df_['release_date'].fillna(df_.groupby('album_id')['year'].transform("first"),inplace=True)
df_['release_date'].fillna(df_.groupby('temp')['release_date'].transform("first"), inplace=True)

In [37]:
df_['year'].fillna(df_.groupby('album_id')['year'].transform("first"), inplace=True)
df_['year'].fillna(df_.groupby('temp')['year'].transform("first"), inplace=True)
df_.drop("temp", axis=1, inplace=True)

In [39]:
df_.drop_duplicates(subset=["name", "primary_artist"], inplace=True)
df_clean = df_[['id','name','primary_artist','secondary_artists','track_genre']][:]

In [40]:
# If a genre is null and the artist/album has made music of a certain genre before, fill with that genre 
df_['track_genre'].fillna(df_.groupby('album_id')['track_genre'].transform("first"), inplace=True)
df_['track_genre'].fillna(df_.groupby('primary_artist_id')['track_genre'].transform("first"), inplace=True)

In [41]:
df_['track_genre'].value_counts()

track_genre
songwriter    34535
classical     12489
german         8468
bluegrass      5782
new-age        5013
              ...  
deep-house      343
happy           330
pagode          301
brazil          245
edm              83
Name: count, Length: 113, dtype: int64

In [42]:
df_['artist_count'] = df_['artists'].apply(lambda x: len(x))
df_.drop(['artists','artist_ids'],axis=1,inplace=True)

In [46]:
df_['explicit'].value_counts()

explicit
False    865939
True      57703
Name: count, dtype: int64

In [48]:
df_['decade'] = df_['year'].apply(lambda x: str(x - (x % 10)))
df_['time_signature'] = df_['time_signature'].apply(lambda x: str(x)+"tsig")

### Compressing categorical data into a single column

In [49]:
df_["content"] = df_.apply(
    lambda x: 
    f'{" ".join([str(x["primary_artist"]).lower().replace(" ", "")] * (2 if x["track_genre"] else 3))} '
    f'{" ".join([str(x["secondary_artists"]).lower().replace(" ", "")] * (1 if x["track_genre"] else 2))} '
    f'{" ".join([str(x["album"]).replace(" ", "").lower()] * 1)} '
    f'{" ".join([str(x["name"]).lower()] * 1)} '
    f'{" ".join([str(x["track_genre"]).replace(" ", "")] * 2)} '
    f'{" ".join([str(x["decade"])] * 2)} '
    f'{" ".join([str(x["language"])] * 1)}'
    f'{" ".join([str(x["time_signature"])] * 2)}',axis=1,
)

df_["content"] = df_["content"].apply(
    lambda x: x.lower().replace(",", "").replace(".", "").replace(":", "").replace("[", "").replace("]","").replace("/", ""))

### Removing more duplicate songs in current dataset

In [50]:
import string
df_['name_upper'] = df_['name'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)).title().replace(" ", ""))
df_['artist_upper'] = df_['primary_artist'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)).title().replace(" ", ""))

In [51]:
print(df_.shape)
df_[df_.duplicated(['name_upper', 'artist_upper'])]
df_ = df_.drop_duplicates(subset=['name_upper', 'artist_upper'], keep='first')
df_.drop(['name_upper', 'artist_upper'], axis=1, inplace=True)
print(df_.shape)

(923642, 31)
(918039, 29)


In [52]:
num_cols = list(df_.select_dtypes(include=[np.number]).columns)
cat_cols = list(df_.select_dtypes(include=['object']).columns)

In [53]:
df_cat = df_[cat_cols]
df_num = df_[num_cols]

In [54]:
df_cat.shape, df_num.shape

((918039, 14), (918039, 14))

In [55]:
ss = StandardScaler()
df_num = pd.DataFrame(ss.fit_transform(df_num), columns=num_cols, index=df_num.index)

In [56]:
df_cat.reset_index(drop=True, inplace=True)
df_num.reset_index(drop=True, inplace=True)

In [57]:
df_m = pd.concat([df_cat, df_num], axis=1)

In [58]:
df_m.isnull().sum()

id                           0
name                         0
album                   118438
album_id                118438
time_signature               0
release_date              6120
track_genre             716208
language                     0
primary_artist               0
secondary_artists            0
primary_artist_id            0
secondary_artist_ids         0
decade                       0
content                      0
danceability                 0
energy                       0
key                          0
loudness                     0
mode                         0
speechiness                  0
acousticness                 0
instrumentalness             0
liveness                     0
valence                      0
tempo                        0
duration_ms                  0
year                      6120
artist_count                 0
dtype: int64

In [59]:
df_m.to_csv("dataset_final.csv", index=False)